In [1]:
import os
import re
import time  


start_time = time.perf_counter()

end_time = time.perf_counter()

elapsed_time_microseconds = (end_time - start_time) * 1e6  # Convert to microseconds

print(f"Elapsed time: {elapsed_time_microseconds:.2f} microseconds")


current_directory = os.getcwd()

relative_folder_path = 'SQL_Files/View'


folder_path = os.path.join(current_directory, relative_folder_path)

try:
    files = os.listdir(folder_path)

    sql_files = [file for file in files if file.endswith('.sql')]

    sql_contents_list = []
    for sql_file in sql_files:
        file_path = os.path.join(folder_path, sql_file)
        with open(file_path, 'r') as file:
            sql_contents = file.read()
            sql_contents_list.append(sql_contents)
            
except FileNotFoundError:
    print(f"Folder not found: {folder_path}")

except Exception as e:
    print(f"An error occurred: {e}")
    
import re
def remove_outer_quotes(sql):
    ls1 = sql.split("(")[0].replace('"','')
    ls2 = ["("+i for i in sql.split("(")[1:]] 
    ls2.insert(0,ls1)
    sql = "".join(ls2)  
    
    return sql
resource_table_name_list = []
def python_terraform(sql):
    code = ""
    ddl = sql.split(';')

    for command in ddl: 
        command = command.strip().upper()
        
        info_pattern = r'CREATE\s+(?:OR\s+REPLACE\s+)?(?:SECURE\s+)?(?:MATERIALIZED\s+)?VIEW\s+(?:(?:"?)([A-Z0-9_]+)(?:"?)\.)?(?:"?)([A-Z0-9_]+)(?:"?)\.([A-Z0-9_]+)'

        info_matches = re.findall(info_pattern, command, re.IGNORECASE)
        for match in info_matches:
            database_name = match[0].strip('"')
            schema_name = match[1].strip('"')
            table_name = match[2].strip('"')
#             print(f"Database: {database_name}, Schema: {schema_name}, Table: {table_name}")

            # data_retention_time_in_days_schema = 1

            # set the dynamic database name  / remove dev , prod name

            dynamic_db = ''
            dynamic__main_db =''  
            if  re.search(r'(_PROD|_DEV)', database_name , re.IGNORECASE  ):     
                dynamic_db = re.sub(r'(_PROD|_DEV)', r'_${var.SF_ENVIRONMENT}',  database_name , re.IGNORECASE  )
                dynamic__main_db += re.sub(r'(_PROD|_DEV)', "",  database_name , re.IGNORECASE )
            else:
                dynamic_db += database_name.strip()
                dynamic__main_db += database_name.strip()

            
        #------------------------------------------------------------------------------------------------
            
#             value_pattern = r'CREATE\s+(?:OR\s+REPLACE\s+)?(?:SECURE\s+)?VIEW\s+(?:(?:"?)([A-Z_]+)(?:"?)\.)?(?:"?)([A-Z_]+)(?:"?)\.([A-Z_]+)[\s\S]*?(?<=AS\s)([\s\S]*)$'
            # value_pattern = r'CREATE\s+(?:OR\s+REPLACE\s+)?(?:SECURE\s+)?(?:MATERIALIZED\s+)?VIEW\s+(?:(?:"?)([A-Z_]+)(?:"?)\.)?(?:"?)([A-Z_]+)(?:"?)\.([A-Z_]+)[\s\S]*?(?<=AS\s)([\s\S]*)$'
            value_pattern = r'CREATE\s+(?:OR\s+REPLACE\s+)?(?:SECURE\s+)?(?:MATERIALIZED\s+)?VIEW\s+(?:(?:"?)([A-Z0-9_]+)(?:"?)\.)?(?:"?)([A-Z0-9_]+)(?:"?)\.([A-Z0-9_]+)[\s\S]*?(?<=AS\s)([\s\S]*)$'

            value_matches = re.search(value_pattern, sql, re.IGNORECASE | re.DOTALL)


      
            if value_matches:
                value_matches = value_matches.group(0)

                extracted_code_single_quotes = value_matches
                global extracted_code_replaced

                if  re.search(r'(_PROD|_DEV|_prod|_dev)', extracted_code_single_quotes):     

                    extracted_code_replaced = re.sub(r'(_PROD|_DEV|_prod|_dev)', r'_${var.SF_ENVIRONMENT}', extracted_code_single_quotes)
                else:
                    extracted_code_replaced = extracted_code_single_quotes
            else:
                pass
            
       
            value_pattern_M_S = r'CREATE\s+(?:OR\s+REPLACE\s+)?(SECURE\s+)?(MATERIALIZED\s+)?VIEW\s+(?:(?:"?)([A-Z0-9_]+)(?:"?)\.)?(?:"?)([A-Z0-9_]+)(?:"?)\.([A-Z0-9_]+)[\s\S]*?(?<=AS\s)([\s\S]*)$'

            value_matches_M_S = re.search(value_pattern_M_S, sql, re.IGNORECASE | re.DOTALL)
              


            is_secure = bool(value_matches_M_S.group(1))  # Check if the "SECURE" keyword is captured
            is_materialized = bool(value_matches_M_S.group(2))  # Check if the "MATERIALIZED" keyword is captured
            or_replace_present = bool(re.search(r'\bOR\s+REPLACE\b', sql, re.IGNORECASE | re.DOTALL))  # Check if "OR REPLACE" is present


            if is_secure:
                resource_name_prefix = "snowflake_secure_view"
            elif is_materialized:
                resource_name_prefix = "snowflake_materialized_view"
            else:
                resource_name_prefix = "snowflake_view"

            resource_database_name = f"resource \"{resource_name_prefix}\" \"{dynamic__main_db}_{schema_name}_{table_name}\""

            code += f"{resource_database_name} {{\n"
            code += f"\tdatabase = \"{dynamic_db}\"\n"


            
            resource_table_name_demo = f'{dynamic__main_db}_{schema_name}_{table_name}'
            resource_table_name_list.append(resource_table_name_demo)
            code += f"\tschema = \"{schema_name}\"\n"
            code += f"\tname = \"{table_name}\"\n"
            
            if ('\'\'') in extracted_code_replaced:
                extracted_code_replaced = extracted_code_replaced.replace('\'\'', '\'')
                code += f"\tstatement = <<-SQL\n\t {extracted_code_replaced}\nSQL\n"
                
            else:
                code += f"\tstatement = <<-SQL\n\t {extracted_code_replaced}\nSQL\n"
            
            if or_replace_present:
                code += f"\tor_replace = true \n"
            else:
                code += f"\tor_replace = false \n"
                
            if is_secure or is_materialized:
                code += f"\tis_secure = true \n"
            else :
                code += f"\tis_secure = false \n"
                
            code += "}\n\n"
            
                
    return code    
for sql_contents in sql_contents_list:
    sql_without_quotes = remove_outer_quotes(sql_contents)
    main = python_terraform(sql_without_quotes)
#     print(main)
    # Extract database name and schema name from the SQL content
    extract_schema_database_table = re.search(r'\b(\w+)\.(\w+)\.(\w+)', sql_without_quotes)
    
    info_pattern = r'CREATE\s+(?:OR\s+REPLACE\s+)?(?:SECURE\s+)?(?:MATERIALIZED\s+)?VIEW\s+(?:(?:"?)([A-Z0-9_]+)(?:"?)\.)?(?:"?)([A-Z0-9_]+)(?:"?)\.([A-Z0-9_]+)'
        
        # Find all matches of the pattern in the SQL code
    extract_schema_database_table = re.findall(info_pattern, sql_without_quotes, re.IGNORECASE)
    
    for match in extract_schema_database_table:
        database_name = match[0].strip('"')
        schema_name = match[1].strip('"')
        table_name = match[2].strip('"')
        
        folder_name = ''
        if database_name.startswith('DISC_') or database_name.startswith('APP_DB_') or database_name.startswith('DEDUPE_STAGE_LAYER'):
            folder_name += 'discovery'
        elif database_name.startswith('DW_'):
            folder_name += 'business'
        elif database_name.startswith('DATA_MANAGEMENT'):
            folder_name += 'data_management'
        elif database_name.startswith('ETL_MANAGEMENT_'):
             folder_name += 'audit'
        else :
             pass
        
        if match:
            output_folder = os.path.join(current_directory, 'Terraform_Files',folder_name,'view')
            
            try:
                os.makedirs(output_folder, exist_ok=True)
            except Exception as e:
                print(f"An error occurred while creating the output folder: {e}")

            # Write Terraform code to the appropriate output file
            try:
                
                dynamic_db = ''
                dynamic__main_db =''  
                if  re.search(r'(_PROD|_DEV)', database_name.upper() , re.IGNORECASE  ):     
                    dynamic_db = re.sub(r'(_PROD|_DEV)', r'_${var.SF_ENVIRONMENT}',  database_name.upper() , re.IGNORECASE  )
                    dynamic__main_db += re.sub(r'(_PROD|_DEV)', "",  database_name.upper() , re.IGNORECASE )
                else:
                    dynamic_db += database_name.upper().strip()
                    dynamic__main_db += database_name.upper().strip()
                    
                resource_table_name = f"{dynamic__main_db}_{schema_name}_{table_name}"
                output_filename = os.path.join(output_folder, f"{resource_table_name}.tf")
                with open(output_filename, 'w',encoding='utf8' ) as tf_file:
                    tf_file.write(main)
            except Exception as e:
                print(f"An error occurred while writing the output file: {e}")
        else:
            print("Unable to extract database name and schema name from the SQL content.")

Elapsed time: 48.00 microseconds
